In [1]:
import pandahouse as ph

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

# Задание 1
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
query = """
 SELECT 
    COUNT (st_id) as count_diligent_students
 FROM   
    (SELECT 
        st_id ,
        SUM(correct) as summa
        
    FROM
        default.peas
    
    WHERE  correct  > 0
    
    GROUP BY
        st_id  
    HAVING 
        summa >= 20)
"""
cds = ph.read_clickhouse(query, connection=connection_default)
print('Количество усердных студентов с 30 по 31 октября составило', cds.count_diligent_students[0], 'человек.')

Количество усердных студентов с 30 по 31 октября составило 136 человек.


# Задание 2
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
ARPU; 
ARPAU; 
CR в покупку; 
СR активного пользователя в покупку; 
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике.
Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [4]:
query = """
SELECT
    ARPU_PILOT,
    ARPPU_PILOT,
    CR_ALL_PILOT,
    CR_ACTIV_10_PILOT,
    CR_MATH_PILOT,
    ARPU_CONTROL,
    ARPPU_CONTROL,
    CR_ALL_CONTROL,
    CR_ACTIV_10_CONTROL,
    CR_MATH_CONTROL
    
FROM
    (
    SELECT
        B.PAY_PILOT / A.ALL_COUNT_PILOT AS ARPU_PILOT,
        B.PAY_PILOT / B.COUNT_PAY_PILOT AS ARPPU_PILOT,
        (B.COUNT_PAY_PILOT / A.ALL_COUNT_PILOT) * 100 AS CR_ALL_PILOT,
        (C.COUNT_PILOT_10 / A.ALL_COUNT_PILOT) * 100 AS CR_ACTIV_10_PILOT,
        (D.COUNT_MATH_PILOT / A.ALL_COUNT_PILOT) * 100 AS CR_MATH_PILOT,
        3 AS KEY_METRICS
    FROM 
        (
        SELECT 
            uniq(st_id)  AS ALL_COUNT_PILOT,
            1 AS key_pilot
        FROM
            default.studs
        JOIN 
            default.peas
        ON 
            default.studs.st_id =  default.peas.st_id
        WHERE 
            test_grp = 'pilot') AS A
    JOIN
        (
        SELECT
            SUM(money) as PAY_PILOT,
            COUNT(st_id) as COUNT_PAY_PILOT,
            1 AS key_pilot
        FROM
            default.studs
        JOIN
            default.final_project_check
        ON
            default.studs.st_id =  default.final_project_check.st_id
        WHERE 
            test_grp = 'pilot') AS B
    ON
        A.key_pilot = B.key_pilot
    JOIN
        (
        SELECT
            COUNT(A.st_id) AS COUNT_PILOT_10,
            1 AS key_pilot
        FROM 
            (SELECT
               A.st_id,
               sum(A.correct) AS correct_sum
            FROM
                default.peas AS A
            LEFT JOIN
                default.final_project_check AS B
            ON
                A.st_id  = B.st_id 
            JOIN
                default.studs as C
            ON 
                A.st_id  = C.st_id 
            WHERE 
                C.test_grp = 'pilot' AND B.money > 0
            GROUP BY 
                A.st_id
            HAVING
                correct_sum > 10)    
        ) AS C
    ON
        A.key_pilot = C.key_pilot
    JOIN 
        (
        SELECT 
            COUNT (A.st_id) AS COUNT_MATH_PILOT,
            1 AS key_pilot
        FROM
            (SELECT
                A.st_id,
                sum(A.correct) AS correct_sum
            FROM
                    default.peas AS A
            LEFT JOIN
                default.final_project_check AS B
            ON
                A.st_id  = B.st_id 
            JOIN
                default.studs as C
            ON 
                A.st_id  = C.st_id 
            WHERE 
                 C.test_grp = 'pilot' AND A.subject = 'Math' AND B.money > 0
            GROUP BY
                A.st_id
            HAVING 
                correct_sum >=2)
        ) AS D
    ON
        A.key_pilot = D.key_pilot    
    )AS A

JOIN
    
    (
    SELECT
        B.PAY_CONTROL / A.ALL_COUNT_CONTROL AS ARPU_CONTROL,
        B.PAY_CONTROL / B.COUNT_PAY_CONTROL AS ARPPU_CONTROL,
        (B.COUNT_PAY_CONTROL / A.ALL_COUNT_CONTROL) * 100 AS CR_ALL_CONTROL,
        (C.COUNT_CONTROL_10 / A.ALL_COUNT_CONTROL) * 100 AS CR_ACTIV_10_CONTROL,
        (D.COUNT_MATH_CONTROL / A.ALL_COUNT_CONTROL) * 100 AS CR_MATH_CONTROL,
        3 AS KEY_METRICS
    FROM 
        (
        SELECT 
            uniq(st_id)  AS ALL_COUNT_CONTROL,
            2 AS key_control
        FROM
            default.studs
        JOIN 
            default.peas
        ON 
            default.studs.st_id =  default.peas.st_id
        WHERE 
            test_grp = 'control') AS A
    JOIN
        (
        SELECT
            SUM(money) as PAY_CONTROL,
            COUNT(st_id) as COUNT_PAY_CONTROL,
            2 AS key_control
        FROM
            default.studs
        JOIN
            default.final_project_check
        ON
            default.studs.st_id =  default.final_project_check.st_id
        WHERE 
            test_grp = 'control') AS B
    ON
        A.key_control = B.key_control
    JOIN
        (
        SELECT
            COUNT(A.st_id) AS COUNT_CONTROL_10,
            2 AS key_control
        FROM 
            (SELECT
               A.st_id,
               sum(A.correct) AS correct_sum
            FROM
                default.peas AS A
            LEFT JOIN
                default.final_project_check AS B
            ON
                A.st_id  = B.st_id 
            JOIN
                default.studs as C
            ON 
                A.st_id  = C.st_id 
            WHERE 
                C.test_grp = 'control' AND B.money > 0
            GROUP BY 
                A.st_id
            HAVING
                correct_sum > 10)    
        ) AS C
    ON
        A.key_control = C.key_control
    JOIN 
        (
        SELECT 
            COUNT (A.st_id) AS COUNT_MATH_CONTROL,
            2 AS key_control
        FROM
            (SELECT
                A.st_id,
                sum(A.correct) AS correct_sum
            FROM
                    default.peas AS A
            LEFT JOIN
                default.final_project_check AS B
            ON
                A.st_id  = B.st_id 
            JOIN
                default.studs as C
            ON 
                A.st_id  = C.st_id 
            WHERE 
                 C.test_grp = 'control' AND A.subject = 'Math' AND B.money > 0
            GROUP BY
                A.st_id
            HAVING 
                correct_sum >=2)
        ) AS D
    ON
        A.key_control = D.key_control    
    ) AS B

ON
    A.KEY_METRICS = B.KEY_METRICS

"""
metrics = ph.read_clickhouse(query, connection=connection_default)
metrics.T

,0
ARPU_PILOT,23741.258741
ARPPU_PILOT,78953.488372
CR_ALL_PILOT,30.069930
CR_ACTIV_10_PILOT,18.181818
CR_MATH_PILOT,4.195804
ARPU_CONTROL,8393.939394
ARPPU_CONTROL,76944.444444
CR_ALL_CONTROL,10.909091
CR_ACTIV_10_CONTROL,8.484848
CR_MATH_CONTROL,3.030303


###### Преобразуем в более четабельный фрейм

In [5]:
def rename_col(df):
    df = df.rename(columns={df.columns[0]: 'arpu', df.columns[1]: 'arppu', df.columns[2]: 'cr_all',
                   df.columns[3]: 'cr_activ_10', df.columns[4]: 'cr_math'})
    return df

metrics = round(rename_col(metrics[metrics.columns[0:5]].rename(index={0: "pilot"}))
                .append(rename_col(metrics[metrics.columns[5:10]].rename(index={0: "control"}))), 2)

metrics

,arpu,arppu,cr_all,cr_activ_10,cr_math
pilot,23741.26,78953.49,30.07,18.18,4.20
control,8393.94,76944.44,10.91,8.48,3.03


##### Глядя на получившиеся результаты, можно предположить, что пилотная группая показывает себя лучше.